In [ ]:
!pip3 install econml

In [ ]:
!pip3 install lightgbm

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

import dowhy
from dowhy import CausalModel

from scipy import stats
import statsmodels.api as sm

from itertools import combinations

NOISE_LEVEL = .2
N_SAMPLES = 1000

# Generate the data
a = np.random.randn(N_SAMPLES) 
x = a + NOISE_LEVEL*np.random.randn(N_SAMPLES) 
b = x + NOISE_LEVEL*np.random.randn(N_SAMPLES)
y = a + x + b + NOISE_LEVEL*np.random.randn(N_SAMPLES)
c = y + b + NOISE_LEVEL*np.random.randn(N_SAMPLES)

# Get combinations
combs = list(combinations([('a', a), ('b', b), ('c', c), ('x', x), ('y', y)], 2))

data = pd.DataFrame(np.vstack([a, b, x, c, y]).T, columns=['A', 'B', 'X', 'C', 'Y'])

data.head()

In [ ]:
nodes = ['A', 'B', 'X', 'C', 'Y']
edges = [
    ('A', 'X'),
    ('X', 'B'),
    ('A', 'Y'),
    ('X', 'Y'),
    ('B', 'Y'),
    ('Y', 'C'),
    ('B', 'C')
]

gml_string = 'graph [directed 1\n'

for node in nodes:
    gml_string += f'\tnode [id "{node}" label "{node}"]\n'

for edge in edges:
    gml_string += f'\tedge [source "{edge[0]}" target "{edge[1]}"]\n'
    
gml_string += ']'

model = CausalModel(
    data=data,
    treatment='X',
    outcome='Y',
    graph=gml_string
)

model.view_model()

In [ ]:
estimand = model.identify_effect()

print(estimand)

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression



estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    method_params={
        'init_params': {
            'model_y': LGBMRegressor(n_estimators=500, max_depth=10),
            'model_t': LogisticRegression(),
            'discrete_treatment': False,
            'cv': 4
        },
        'fit_params': {}
    })

estimate.cate_estimates.mean()